# Reading all the pdfs and converting them to documents

In [15]:
import os 
from langchain_community.document_loaders import PyPDFLoader 
docs = []
for pdf in os.listdir('pdfs'):
    loader = PyPDFLoader('pdfs/{pdf_file}'.format(pdf_file = pdf))
    for document in loader.load():
        docs.append(document)


In [16]:
docs

[Document(metadata={'source': 'pdfs/Attention is all you need.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architectur

# Chunking

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 20)
documents = text_splitter.split_documents(docs)

In [19]:
URL = "Add the public url you got from running Running_Ollama.ipynb"

# Creating A Faiss Vector Database

In [22]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents,OllamaEmbeddings(base_url = URL))

Loading LLAMA2 Model

In [23]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2" , base_url = URL)
llm

Ollama(base_url='https://able-mayfly-driven.ngrok-free.app')

Prompt

In [24]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

Combine Document Chain

In [25]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

Retriver chain

In [26]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E1CBD16870>, search_kwargs={})

In [27]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

# Run Gradio Interface

In [35]:
import gradio 

def answer(text_input):
    return retrieval_chain.invoke({"input":text_input})['answer']

demo = gradio.Interface(fn = answer , inputs = 'text' , outputs = 'text')
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Stopping gradio

In [38]:
demo.close()

Closing server running on port: 7861
